### Extract Rooms from IFC and plot them

In [1]:
import ifcopenshell
import re
import networkx as nx
import pandas as pd
import plotly.graph_objs as go
import csv
from plotly.subplots import make_subplots

def load_ifc_file(file_path):
    """Load the IFC file."""
    return ifcopenshell.open(file_path)

def filter_ifcspaces_by_storey(spaces, storey_name):
    """Filter IfcSpaces by the given storey name."""
    filtered_spaces = []
    for space in spaces:
        for rel in space.Decomposes:
            if rel.is_a("IfcRelAggregates") and rel.RelatingObject.is_a("IfcBuildingStorey"):
                if rel.RelatingObject.Name == storey_name:
                    filtered_spaces.append(space)
                    break
    return filtered_spaces

def filter_spaces_by_name(spaces):
    """Filter IfcSpaces by name containing only digits (1-5 digits)."""
    digit_pattern = re.compile(r'^\d{1,5}$')
    filtered_spaces = []
    for space in spaces:
        if digit_pattern.match(space.Name):
            filtered_spaces.append(space)
    return filtered_spaces

# -- MAIN --

# Load the IFC file
ifc_file = load_ifc_file("Hus28_test.ifc")
print("IFC file opened successfully.")

# Define the storey name to filter
storey_name = "Plan 10"

# Find all IfcSpaces
all_spaces = ifc_file.by_type("IfcSpace")
num_spaces = len(all_spaces)
print(f"Number of IfcSpaces found: {num_spaces}")

# Filter spaces by the specified storey name
spaces = filter_ifcspaces_by_storey(all_spaces, storey_name)
num_filtered_spaces = len(spaces)
print(f"Number of IfcSpaces matching storey '{storey_name}': {num_filtered_spaces}")

# Further filter spaces by names containing only digits (1-5 digits)
spaces = filter_spaces_by_name(spaces)
num_filtered_by_name = len(spaces)
print(f"Number of IfcSpaces with valid names: {num_filtered_by_name}")

# Create a NetworkX graph
G = nx.Graph()

# Add nodes to the graph
for space in spaces:
    node_label = f"Room_{space.Name}"
    G.add_node(node_label, color='yellow')

# Create the Plotly graph
pos = nx.spring_layout(G)  # Positions for all nodes

# Extract node positions
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_text = list(G.nodes())

# Create node trace
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=node_text,
    textposition='top center',
    hoverinfo='text',
    marker=dict(
        color='yellow',
        size=20,
        line=dict(width=2)
    )
)

# Create edge trace
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=1, color='#888'),
    hoverinfo='none',
    mode='lines'
)

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Network graph of IFC Spaces',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

# Show the figure
fig.show()


IFC file opened successfully.
Number of IfcSpaces found: 193
Number of IfcSpaces matching storey 'Plan 10': 37
Number of IfcSpaces with valid names: 30


### Add direct Contact between Rooms

In [2]:
# Load the CSV file
csv_file = "Output01_RoomsBySeparationLine.csv"

# Manually read and parse the CSV file
edges = []
with open(csv_file, 'r') as file:
    for line in file:
        rooms = line.strip().split(';')
        main_room = rooms[0]
        for neighbor in rooms[1:]:
            edges.append((main_room, neighbor))

# Add edges to the graph
for edge in edges:
    room1 = f"Room_{edge[0]}"
    room2 = f"Room_{edge[1]}"
    if room1 in G and room2 in G:
        G.add_edge(room1, room2, Access="Direct")

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color='blue' if edge[2]['Access'] == 'Direct' else 'black'),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color='yellow', size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


### Add Contact via Doors

In [3]:
# Load the second CSV file
csv_file_doors = "Output02_RoomsByDoors.csv"

# Manually read and parse the second CSV file
door_edges = []
with open(csv_file_doors, 'r') as file:
    for line in file:
        rooms = line.strip().split(';')
        if len(rooms) > 3:
            print(f"Door {rooms[0]} connects more than two rooms: {rooms[1:]}")
        if len(rooms) > 1:
            for i in range(1, len(rooms) - 1):
                for j in range(i + 1, len(rooms)):
                    door_edges.append((rooms[i], rooms[j]))

# Add door edges to the graph
for edge in door_edges:
    room1 = f"Room_{edge[0]}"
    room2 = f"Room_{edge[1]}"
    if room1 in G and room2 in G:
        G.add_edge(room1, room2, Access="Door")

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2]['Access'] == 'Direct' else 'green' if edge[2]['Access'] == 'Door' else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color='yellow', size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()



Door 36670 connects more than two rooms: ['750', '750', '928']


### Add Contact via Windows

In [4]:
# Load the third CSV file
csv_file_windows = "Output04_RoomsByWindows.csv"

# Manually read and parse the third CSV file
window_edges = set()
with open(csv_file_windows, 'r') as file:
    for line in file:
        rooms = line.strip().split(';')[1:]
        if len(rooms) > 1:
            for i in range(len(rooms)):
                for j in range(i + 1, len(rooms)):
                    edge = (f"Room_{rooms[i]}", f"Room_{rooms[j]}")
                    # Ensure bidirectional uniqueness
                    if edge[0] != edge[1] and (edge not in window_edges and (edge[1], edge[0]) not in window_edges):
                        window_edges.add(edge)

# Add window edges to the graph
for room1, room2 in window_edges:
    if room1 in G and room2 in G:
        G.add_edge(room1, room2, Access="Window")

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2]['Access'] == 'Direct' else 'green' if edge[2]['Access'] == 'Door' else 'red' if edge[2]['Access'] == 'Window' else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color='yellow', size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


### Add Doors

In [5]:
# Load the second CSV file
csv_file_doors = "Output02_RoomsByDoors.csv"

# Manually read and parse the second CSV file
door_edges = []
doors = set()
with open(csv_file_doors, 'r') as file:
    for line in file:
        rooms = line.strip().split(';')
        door = rooms[0]
        doors.add(door)
        if len(rooms) > 3:
            print(f"Door {door} connects more than two rooms: {rooms[1:]}")
        if len(rooms) > 1:
            for room in rooms[1:]:
                door_edges.append((door, room))

# Add door nodes to the graph
for door in doors:
    G.add_node(f"Door_{door}", color='blue')

# Add door edges to the graph
for edge in door_edges:
    door = f"Door_{edge[0]}"
    room = f"Room_{edge[1]}"
    if room in G:
        G.add_edge(door, room, IsConnected=True)

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2].get('Access') == 'Direct' else 'orange' if edge[2].get('Access') == 'Door' else 'red' if edge[2].get('IsConnected') else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


Door 36670 connects more than two rooms: ['750', '750', '928']


### Add Windows

In [6]:
# Load the third CSV file
csv_file_windows = "Output04_RoomsByWindows.csv"

# Manually read and parse the third CSV file
window_edges = []
windows = set()
with open(csv_file_windows, 'r') as file:
    for line in file:
        rooms = line.strip().split(';')
        window = rooms[0]
        windows.add(window)
        if len(rooms) > 3:
            print(f"Window {window} connects more than two rooms: {rooms[1:]}")
        if len(rooms) > 1:
            for room in rooms[1:]:
                window_edges.append((window, room))

# Add window nodes to the graph
for window in windows:
    G.add_node(f"Window_{window}", color='purple')

# Add window edges to the graph
for edge in window_edges:
    window = f"Window_{edge[0]}"
    room = f"Room_{edge[1]}"
    if room in G:
        G.add_edge(window, room, IsConnected=True)

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2].get('Access') == 'Direct' else 'orange' if edge[2].get('Access') == 'Door' else 'purple' if edge[2].get('IsConnected') else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


Window 104474 connects more than two rooms: ['19', '19', '922', '922']
Window 104459 connects more than two rooms: ['21', '21', '1104', '1104']
Window 100806 connects more than two rooms: ['27', '27', '1108', '1108']
Window 100821 connects more than two rooms: ['28', '28', '926', '926']


## Add Walls

In [7]:
def get_storey(element):
    """Get the storey of the given element."""
    for rel in element.ContainedInStructure:
        if rel.is_a("IfcRelContainedInSpatialStructure") and rel.RelatingStructure.is_a("IfcBuildingStorey"):
            return rel.RelatingStructure
    return None

# Find all IfcWalls
all_walls = ifc_file.by_type("IfcWall")
num_walls = len(all_walls)
print(f"Number of IfcWalls found: {num_walls}")

# Filter walls by the specified storey name
wall_oids_in_storey = []
for wall in all_walls:
    storey = get_storey(wall)
    if storey and storey.Name == storey_name:
        wall_oids_in_storey.append(wall.id())
num_filtered_walls = len(wall_oids_in_storey)
print(f"Number of IfcWalls matching storey '{storey_name}': {num_filtered_walls}")

# Add wall nodes to the graph
for wall_oid in wall_oids_in_storey:
    node_label = f"Wall_{wall_oid}"
    G.add_node(node_label, color='grey')

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2].get('Access') == 'Direct' else 'orange' if edge[2].get('Access') == 'Door' else 'purple' if edge[2].get('IsConnected') else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()



Number of IfcWalls found: 638
Number of IfcWalls matching storey 'Plan 10': 138


### Connection Room-Walls

In [8]:
# Helper function to get wall OID from GUID
def get_wall_oid_by_guid(walls, guid):
    for wall in walls:
        if wall.GlobalId == guid:
            return wall.id()
    return None

# Load the CSV file
csv_file_spaces_walls = "Output06_Spaces_Walls.csv"

# Manually read and parse the CSV file
with open(csv_file_spaces_walls, 'r') as file:
    reader = csv.DictReader(file, delimiter=';')
    for row in reader:
        space_name = row['Space Name']
        space_node = f"Room_{space_name}"
        wall_guids = row['Wall GUIDs'].split(',')

        for wall_guid in wall_guids:
            wall_oid = get_wall_oid_by_guid(all_walls, wall_guid)
            if wall_oid is not None:
                wall_node = f"Wall_{wall_oid}"
                if space_node in G and wall_node in G:
                    G.add_edge(space_node, wall_node, ContainedIn=True)

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = 'blue' if edge[2].get('Access') == 'Direct' else 'orange' if edge[2].get('Access') == 'Door' else 'purple' if edge[2].get('Access') == 'Window' else 'grey' if edge[2].get('ContainedIn') else 'black'
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


### Connections Walls - Walls

In [9]:
# Load the CSV file
csv_file_walls_to_walls = "Output03_WallsToWalls.csv"

# Helper function to get wall OID from GUID
def get_wall_oid_by_guid(walls, guid):
    for wall in walls:
        if wall.GlobalId == guid:
            return wall.id()
    return None

# Manually read and parse the CSV file
with open(csv_file_walls_to_walls, 'r') as file:
    reader = csv.reader(file, delimiter=';')
    for row in reader:
        primary_wall_guid = row[0]
        primary_wall_oid = get_wall_oid_by_guid(all_walls, primary_wall_guid)
        if primary_wall_oid is not None:
            primary_wall_node = f"Wall_{primary_wall_oid}"
            for connected_wall_guid in row[1:]:
                connected_wall_oid = get_wall_oid_by_guid(all_walls, connected_wall_guid)
                if connected_wall_oid is not None:
                    connected_wall_node = f"Wall_{connected_wall_oid}"
                    if primary_wall_node in G and connected_wall_node in G:
                        G.add_edge(primary_wall_node, connected_wall_node, ConnectedWith=True)

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = ('blue' if edge[2].get('Access') == 'Direct' else
             'orange' if edge[2].get('Access') == 'Door' else
             'purple' if edge[2].get('Access') == 'Window' else
             'grey' if edge[2].get('ContainedIn') else
             'red' if edge[2].get('ConnectedWith') else
             'black')
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()


### Host of Windows and Doors

In [10]:
import csv

# Helper function to get element OID from GUID
def get_element_oid_by_guid(elements, guid):
    for element in elements:
        if element.GlobalId == guid:
            return element.id()
    return None

# Load the CSV file
csv_file_host_elements = "Output05_HostWindowsAndDoors.csv"

# Manually read and parse the CSV file
with open(csv_file_host_elements, 'r') as file:
    reader = csv.DictReader(file, delimiter=';')
    for row in reader:
        element_type = row['Element Type']
        element_guid = row['Element GUID']
        wall_oid = row['Wall OID']

        element_oid = get_element_oid_by_guid(ifc_file.by_type(element_type), element_guid)
        if element_oid is not None:
            element_node = f"{element_type.split('Ifc')[-1]}_{element_oid}"
            wall_node = f"Wall_{wall_oid}"
            
            if element_node in G and wall_node in G:
                G.add_edge(element_node, wall_node, HostedBy=True)

# Calculate node positions
pos = nx.spring_layout(G)

# Prepare nodes and edges for Plotly
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = ('blue' if edge[2].get('Access') == 'Direct' else
             'orange' if edge[2].get('Access') == 'Door' else
             'purple' if edge[2].get('Access') == 'Window' else
             'grey' if edge[2].get('ContainedIn') else
             'red' if edge[2].get('ConnectedWith') else
             'green' if edge[2].get('HostedBy') else
             'black')
    edge_trace.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None],
                                 line=dict(width=1, color=color),
                                 hoverinfo='none', mode='lines'))

node_trace = go.Scatter(x=[pos[node][0] for node in G.nodes], y=[pos[node][1] for node in G.nodes],
                        text=[node for node in G.nodes], mode='markers+text',
                        textposition='top center', hoverinfo='text',
                        marker=dict(showscale=False, color=[G.nodes[node]['color'] for node in G.nodes], size=10, line_width=2))

# Create Plotly figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(showlegend=False, hovermode='closest',
                                 margin=dict(b=20, l=5, r=5, t=40),
                                 xaxis=dict(showgrid=False, zeroline=False),
                                 yaxis=dict(showgrid=False, zeroline=False)))

fig.show()



In [11]:
import plotly.io as pio

# Save Plotly to HTML
pio.write_html(fig, 'graph.html', auto_open=True)

In [12]:
import pickle

# Save NetworkX to pickle
with open('network_graph.pkl', 'wb') as f:
    pickle.dump(G, f)